# Email Reply Assistant with LM Studio
This notebook demonstrates how to connect to a local Language Model (LM) Studio server and use it to generate email replies.

In [ ]:
# Step 1: Install required libraries (if not already installed)
%pip install requests

In [ ]:
# Step 2: Import necessary libraries
import requests

In [ ]:
# Step 3: Define connection details to your local LM Studio server
LM_STUDIO_URL = "http://localhost:1234/v1/chat/completions"

In [ ]:
# Step 4: Function to generate content using LM Studio
def generate_content(prompt, max_tokens=100, temperature=0.7, model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"):
    headers = {
        'Content-Type': 'application/json',
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        result = response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return ""

In [ ]:
# Step 5: Load email content from a text file
FILE_NAME = "../email.txt"
with open(FILE_NAME, 'r') as file:
    incoming_email = file.read()
print("Incoming Email:")
print(incoming_email)

In [ ]:
# Step 6: Analyze the incoming email
analysis_prompt = f"Analyze the following email and extract the key points: {incoming_email}. Keep the output only to the point no addional comments."
analysis = generate_content(analysis_prompt)
print("Analysis:")
print(analysis)

In [ ]:
# Step 7: To Conclude 3 possible intents to reply
format = "Provide a numbered list of options"
intents_prompt = f"Based on the following email, suggest 3 possible intents for a reply. Only provide short sentences expressing the intention, not the full email. \nEmail: {incoming_email}\nAnalysis: {analysis}. {format}"
intents = generate_content(intents_prompt, max_tokens=100)
print("Possible Reply Intents:")
print(intents)

In [ ]:
# Step 7: Select an intent to reply
option = 2
option_prompt = f"Repeat option {option} exactly as it appears in the text below: {intents}. Don't list any other options."
intent = generate_content(option_prompt, max_tokens=100)
print("Selected intent:")
print(intent)

In [ ]:
# Step 8: Generate a response based on the selected intent
language = "Reply in the same language as the incoming email"
response_prompt = f"Generate a polite and professional reply to the following email: {incoming_email}\nAnalysis: {analysis}\n Use following Intent: {intent}.\n {language}"
response = generate_content(response_prompt, max_tokens=500)
print("Generated Response:")
print(response)

In [ ]:
# Step 9: Refine and proofread the generated response
tone = "Keep it short and casual."
language = "Reply in the same language as the incoming email"
refine_prompt = f"Refine and proofread the following email reply: {response}\n{tone}.\n {language}"
refined_response = generate_content(refine_prompt, max_tokens=150)
print("Refined Response:")
print(refined_response)